In [115]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import SGDClassifier

from keras.models import Sequential
from keras.layers import Dense, Dropout

## Data Analysis

In [109]:
df_train = pd.read_csv('train.csv').sample(5000)

## Training

In [110]:
tfidf = TfidfVectorizer(ngram_range=(1, 1),
                        max_features=50000)

In [137]:
X_train = tfidf.fit_transform(df_train['comment_text'])
y_train = df_train.iloc[:,2:].values

In [138]:
print X_train.shape, y_train.shape

(5000, 23606) (5000, 6)


### Sklearn multi model

In [139]:
class_names = ['toxic', 'severe_toxic', 'obscene',
               'threat', 'insult', 'identity_hate']

clfs = {}
roc_aucs = []

for i in range(5):
    print 'fitting "{0}" classifier'.format(class_names[i])
    clf = SGDClassifier(loss='log')
    score =  np.mean(cross_val_score(clf,
                                     X_train,
                                     y_train[:,i],
                                     cv=3,
                                     scoring='roc_auc'))
    
    clf.fit(X_train, y_train[:, i])
    clfs[class_names[i]] = clf
    roc_aucs.append(score)
    

fitting "toxic" classifier
fitting "severe_toxic" classifier
fitting "obscene" classifier
fitting "threat" classifier
fitting "insult" classifier


In [140]:
np.mean(roc_aucs)

0.9440618622518272

## Test

In [141]:
df_test = pd.read_csv('test.csv')
X_test = tfidf.transform(df_test['comment_text'])

In [154]:
preds = []

for class_name, model in clfs.items():
    model.predict_proba(X_test)
    
preds

(153164, 2)
(153164, 2)
(153164, 2)
(153164, 2)
(153164, 2)


[]

In [150]:
submission_df = pd.read_csv('sample_submission.csv')
submission_df.iloc[:,1:] = y_pred

ValueError: Must have equal len keys and value when setting with an iterable

In [ ]:
submission_df.head()